<a href="https://colab.research.google.com/github/william-toscani/World-Energy-Consumption-Project/blob/main/Project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries

In [ ]:
!pip install pandasql

import requests
import pandas as pd
import numpy as np
import plotly.express as px
from pandasql import sqldf

sql = lambda q: sqldf(q, globals())

# Datasets (data sources)

[1. Our World In Data API (OWID API)](https://ourworldindata.org/)
- [1.1 Population](https://ourworldindata.org/grapher/population-with-un-projections?tab=table)
- [1.2 Electricity production by source](https://ourworldindata.org/grapher/electricity-prod-source-stacked)
- [Share of electricity production by source](https://ourworldindata.org/grapher/share-elec-by-source)
- [CO₂ emissions](https://ourworldindata.org/grapher/annual-co2-emissions-per-country?country=USA~GBR~IND~CHN)
- [Per capita electricity generation by source](https://ourworldindata.org/grapher/per-capita-electricity-source-stacked?country=OWID_WRL~CAN~SWE~USA~GBR~FRA)
- [Global primary energy consumption by source](https://ourworldindata.org/grapher/global-energy-substitution)
[]()


[2. Terna API](https://developer.terna.it/#it)

# Italy in c02 emissions

In [ ]:
# Fetch the data and metadata
co2_emissions_df = pd.read_csv("https://ourworldindata.org/grapher/co-emissions-per-capita.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
co2_emissions_metadata = requests.get("https://ourworldindata.org/grapher/co-emissions-per-capita.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

In [ ]:
pd.set_option('display.max_rows', None)
display(co2_emissions_df.head())
display(co2_emissions_metadata)

In [ ]:
# QUERY: top 10 countries with larger emissions_total_per_capita and italy

emission_df = sql("""
SELECT code, year, emissions_total_per_capita
FROM (
    SELECT code, year, emissions_total_per_capita
    FROM co2_emissions_df
    WHERE year = 2024
    ORDER BY emissions_total_per_capita DESC
    LIMIT 10
) AS top_countries

UNION ALL

SELECT code, year, emissions_total_per_capita
FROM co2_emissions_df
WHERE code = 'ITA' AND year = 2024
ORDER BY emissions_total_per_capita DESC
""")
display(emission_df)


In [ ]:
fig = px.treemap(emission_df, path=['code'], values='emissions_total_per_capita', width=600, height=600, title='CO2 Emissions per Capita: Top 10 Countries and Italy (2024)')
fig.update_traces(textposition='middle center')
fig.show()

In [ ]:
country_continent_df = pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv')

# Select relevant columns and rename for merging
country_continent_df = country_continent_df[['alpha-3', 'region']].rename(columns={'alpha-3': 'code', 'region': 'continent'})

# Display the first few rows of the continent data
display(country_continent_df.head())

### Merge Data and Update Treemap

Now, I will merge the `emission_df` with the `country_continent_df` on the `code` column, and then generate the treemap with the new hierarchical path `['continent', 'code']`.

In [ ]:
# Merge emission data with continent information
emission_continent_df = pd.merge(emission_df, country_continent_df, on='code', how='left')

# Handle potential missing continents for codes not found in the external dataset (e.g., 'SXM', 'PLW', 'NCL', 'FRO', 'CUW')
# For simplicity, I'll fill missing continents with 'Unknown' or a placeholder if they are not in the external dataset.
# A more robust solution might involve manually mapping these or using another dataset.
# Let's check which codes might be missing:
# missing_codes = emission_continent_df[emission_continent_df['continent'].isnull()]['code'].unique()
# print(f"Codes with missing continent information: {missing_codes}")

emission_continent_df['continent'] = emission_continent_df['continent'].fillna('Unknown or Special Territory')

# Display the merged DataFrame
display(emission_continent_df.head())

# Create the treemap with continent hierarchy
fig = px.treemap(emission_continent_df, path=['continent', 'code'], values='emissions_total_per_capita', width=600, height=600, title='CO2 Emissions per Capita by Continent and Country (2024)')
fig.update_traces(textposition='middle center')
fig.show()

In [ ]:
population_df = pd.read_csv("https://ourworldindata.org/grapher/population-with-un-projections.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
population_metadata = requests.get("https://ourworldindata.org/grapher/population-with-un-projections.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

In [ ]:
display(population_df.head())

In [ ]:
import pandas as pd
import requests

# Fetch the data.
df = pd.read_csv("https://ourworldindata.org/grapher/annual-co2-emissions-per-country.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})

# Fetch the metadata
metadata = requests.get("https://ourworldindata.org/grapher/annual-co2-emissions-per-country.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()